In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow import keras
import tensorflow as tf
import os
import time
import numpy as np
import pickle
from sklearn.model_selection import RandomizedSearchCV
import scikeras
from scikeras.wrappers import KerasRegressor

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

In [3]:
es = tf.keras.callbacks.EarlyStopping(patience=10,min_delta=1.0,verbose=1)

In [4]:
def get_callback(filename,name,value):
    root_logdir = os.path.join(os.curdir, filename)
    ts = int(time.time())
    filen = str(ts)+'_'+str(name)+'_'+str(value)
    return tf.keras.callbacks.TensorBoard(os.path.join(root_logdir, filen))

In [5]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate, momentum=0):
    model = tf.keras.models.Sequential()
    model.add(keras.layers.Input(shape=13))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    if optimizer == 'sgd': 
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=False)
    elif optimizer == 'nesterov':
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=True)
    elif optimizer == 'momentum':
        opt = keras.optimizers.SGD(lr=learning_rate,nesterov=False,momentum=momentum)
    elif optimizer == 'adam':
        opt = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='mse',optimizer=opt,metrics=['mse','mae'])
    return model

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
learning = [0.000001,0.00001,0.0001]
lr = []
for rate in learning:
    model = build_model(1,25,'sgd',rate)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','lr',rate),es],verbose=0)
    result = (rate, history.history['loss'][-1], history.history['mae'][-1])
    lr.append(result)
lr

Metal device set to: Apple M1


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 25: early stopping


[(1e-06, 403.6007385253906, 16.933063507080078),
 (1e-05, 324.1733093261719, 14.382200241088867),
 (0.0001, 517.9978637695312, 20.347984313964844)]

In [8]:
hidden = [0,1,2,3]
hl = []
for hid in hidden:
    model = build_model(hid,25,'sgd',1e-05)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','hl',hid),es],verbose=0)
    result = (hid, history.history['loss'][-1], history.history['mae'][-1])
    hl.append(result)
hl

Epoch 11: early stopping
Epoch 87: early stopping
Epoch 75: early stopping


[(0, 23378.021484375, 72.62944793701172),
 (1, 304.5304870605469, 13.697992324829102),
 (2, 283.9238586425781, 12.768856048583984),
 (3, 325.1810607910156, 14.043087005615234)]

In [9]:
neurons = [5,25,125]
nn = []
for neuron in neurons:
    model = build_model(1,neuron,'sgd',1e-05)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','nn',neuron),es],verbose=0)
    result = (neuron, history.history['loss'][-1], history.history['mae'][-1])
    nn.append(result)
nn

[(5, 188.4024658203125, 9.955838203430176),
 (25, 307.8041687011719, 13.89139175415039),
 (125, 304.9493713378906, 13.756514549255371)]

In [10]:
opti = ['sgd','nesterov','momentum','adam']
opt = []
for optimizer in opti:
    model = build_model(1,25,optimizer,1e-05,0.5)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','opt',optimizer),es],verbose=0)
    result = (optimizer, history.history['loss'][-1], history.history['mae'][-1])
    opt.append(result)
opt

Epoch 82: early stopping
Epoch 81: early stopping


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[('sgd', 381.6230163574219, 16.060335159301758),
 ('nesterov', 328.0186767578125, 14.635824203491211),
 ('momentum', 414.0743713378906, 16.966266632080078),
 ('adam', 10000.6484375, 67.87552642822266)]

In [11]:
moment = [0.1,0.5,0.9]
mom = []
for momentum in moment:
    model = build_model(1,25,'momentum',1e-05,momentum)
    history = model.fit(X_train,y_train,epochs=100,validation_split=0.1,callbacks=[get_callback('tb_logs','mom',momentum),es],verbose=0)
    result = (momentum, history.history['loss'][-1], history.history['mae'][-1])
    mom.append(result)
mom

Epoch 82: early stopping
Epoch 25: early stopping


[(0.1, 319.1545715332031, 14.17618465423584),
 (0.5, 315.14849853515625, 13.919328689575195),
 (0.9, 555.3201904296875, 21.264873504638672)]

In [12]:
with open('lr.pkl','wb') as f:
    pickle.dump(lr,f)
with open('hl.pkl','wb') as f:
    pickle.dump(hl,f)
with open('nn.pkl','wb') as f:
    pickle.dump(nn,f)
with open('opt.pkl','wb') as f:
    pickle.dump(opt,f)
with open('mom.pkl','wb') as f:
    pickle.dump(mom,f)

In [13]:
param_distribs = {
"model__n_hidden": [0,1,2,3],
"model__n_neurons": [5,25,125],
"model__learning_rate": [0.000001,0.00001,0.0001],
"model__optimizer": ['sgd','nesterov', 'momentum','adam'],
"model__momentum": [0.1,0.5,0.9]
}

In [14]:
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [ ]:
rnd_search_cv = RandomizedSearchCV(keras_reg,param_distribs,n_iter=30,cv=3,verbose=1)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)

INFO:tensorflow:Assets written to: ram:///var/folders/zz/zyxvpxvq6csfxvn_n00000s4000069/T/tmpkgke2r11/assets
Fitting 3 folds for each of 30 candidates, totalling 90 fits
INFO:tensorflow:Assets written to: ram:///var/folders/zz/zyxvpxvq6csfxvn_n00000s4000069/T/tmpext5n_tu/assets
Epoch 1/100
1/8 [==>...........................] - ETA: 1s - loss: 1268.4097 - mse: 1268.4097 - mae: 26.3360

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


8/8 [==============================] - 0s 20ms/step - loss: 519.9725 - mse: 519.9725 - mae: 18.1849 - val_loss: 251.1025 - val_mse: 251.1025 - val_mae: 13.0880
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 332.2417 - mse: 332.2417 - mae: 14.4841 - val_loss: 218.4224 - val_mse: 218.4224 - val_mae: 11.7252
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 310.1374 - mse: 310.1374 - mae: 13.7093 - val_loss: 216.3014 - val_mse: 216.3014 - val_mae: 11.6033
Epoch 4/100
8/8 [==============================] - 0s 7ms/step - loss: 309.6716 - mse: 309.6716 - mae: 13.6500 - val_loss: 222.7689 - val_mse: 222.7689 - val_mae: 12.0556
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 309.6989 - mse: 309.6989 - mae: 13.6722 - val_loss: 213.3573 - val_mse: 213.3573 - val_mae: 11.3626
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 308.9870 - mse: 308.9870 - mae: 13.6676 - val_loss: 213.9463 - val_mse: 213.9463 - va

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/Guest/.local/lib/python3.9/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/Guest/.local/lib/python3.9/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/opt/homebre

Epoch 1/100


...

8/8 [==============================] - 0s 15ms/step - loss: 23041.8262 - mse: 23041.8262 - mae: 71.6140 - val_loss: 23629.0918 - val_mse: 23629.0918 - val_mae: 73.5344
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 23041.8242 - mse: 23041.8242 - mae: 71.6140 - val_loss: 23629.0918 - val_mse: 23629.0918 - val_mae: 73.5344
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 23041.8242 - mse: 23041.8242 - mae: 71.6140 - val_loss: 23629.0918 - val_mse: 23629.0918 - val_mae: 73.5344
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 23041.8262 - mse: 23041.8262 - mae: 71.6140 - val_loss: 23629.0918 - val_mse: 23629.0918 - val_mae: 73.5344
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 23041.8262 - mse: 23041.8262 - mae: 71.6140 - val_loss: 23629.0918 - val_mse: 23629.0918 - val_mae: 73.5344
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 23041.8262 - mse: 23041.8262 - mae: 71.6140 -

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/Guest/.local/lib/python3.9/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/Guest/.local/lib/python3.9/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/opt/homebre

Epoch 1/100


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


8/8 [==============================] - 0s 22ms/step - loss: 617.8920 - mse: 617.8920 - mae: 21.1290 - val_loss: 348.5784 - val_mse: 348.5784 - val_mae: 16.4508
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 418.8069 - mse: 418.8069 - mae: 17.3975 - val_loss: 291.3195 - val_mse: 291.3195 - val_mae: 14.7326
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 381.9932 - mse: 381.9932 - mae: 16.1968 - val_loss: 271.1105 - val_mse: 271.1105 - val_mae: 13.9825
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 366.9913 - mse: 366.9913 - mae: 15.6508 - val_loss: 259.9752 - val_mse: 259.9752 - val_mae: 13.5741
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 353.4204 - mse: 353.4204 - mae: 15.2211 - val_loss: 248.1113 - val_mse: 248.1113 - val_mae: 13.1732
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 341.5623 - mse: 341.5623 - mae: 14.8655 - val_loss: 240.7906 - val_mse: 240.7906 - va

In [ ]:
rnd_search_cv.best_params_

In [ ]:
with open('rnd_search.pkl','wb') as f:
    pickle.dump(rnd_search_cv.best_params_,f)

In [ ]:
rnd_search_cv.best_score_